In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F #torch是关于运算的包
import torchvision
from torchvision import datasets,transforms, models #torchvision则是打包了一些数据集
from torch.utils.data import Dataset
from torchvision import transforms as T
from torchnet import meter
from torch.autograd import Variable
import pandas as pd
import numpy as np
import glob
from natsort import natsorted
from PIL import Image
import matplotlib.pyplot as plt
import os
from progressbar import * #进度条
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [2]:
#残差块的第一种实现方式，对应于resnet_18
class ResidualBlock_1(nn.Module): ## 继承 torch 的 Module
    def __init__(self, in_channel, out_channel, stride=1):
        super(ResidualBlock_1, self).__init__() #  # 继承 __init__ 功能
        #super() 函数是用于调用父类(超类)的一个方法 调用nn.Module
        self.left = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
            ##inplace为True，将会改变输入的数据 ，否则不会改变原输入，只会产生新的输出
            nn.Conv2d(out_channel, out_channel, kernel_size=3, stride=1, padding=1, bias=False),
            #out_size_w = (input_size_w - kernel_size + 2*padding)/stride + 1 = input_size_w
            nn.BatchNorm2d(out_channel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, stride=stride, bias=False),# padding Default: 0 
                nn.BatchNorm2d(out_channel)
            )
    def forward(self, x):## 这同时也是 Module 中的 forward 功能
        #在pytorch中只需要定义forward函数即可, 反向传播backward的部分在你使用autograd时会自动生成
        out = self.left(x)
        out += self.shortcut(x)
        out = F.relu(out)
        #，nn.Conv2d是一个类，而F.conv2d()是一个函数，
        #而nn.Conv2d的forward()函数实现是用F.conv2d()实现的
        #（在Module类里的__call__实现了forward()函数的调用，
        #所以当实例化nn.Conv2d类时，forward()函数也被执行了
        return out

In [3]:
class ResNet_18(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet_18, self).__init__()
        self.in_channel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock_1, 64, 2, stride=1)#output_w = 32
        self.layer2 = self.make_layer(ResidualBlock_1, 128, 2, stride=2)#output_w = 16
        self.layer3 = self.make_layer(ResidualBlock_1, 256, 2, stride=2)#output_w = 8
        self.layer4 = self.make_layer(ResidualBlock_1, 512, 2, stride=2)#out_put_w = 4
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channel, channels, stride))
            self.in_channel = channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [4]:
#残差块的第二种实现方式 对应于resnet_34
class ResidualBlock_2(nn.Module):
    def __init__(self, in_channel, out_channel, stride=1, shortcut=None):
        super(ResidualBlock_2, self).__init__()
        self.left=nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_channel),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channel, out_channel, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channel)
        )
        self.right=shortcut
    def forward(self, x):
        out=self.left(x)
        residual=x if self.right is None else self.right(x)
        out+=residual
        return F.relu(out)

In [5]:
class ResNet_34(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet_34, self).__init__()
        #前几层图像转换
        self.pre = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),#[112, 112, 64]
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(3, 2, 1)#[56, 56, 64] (112 + 1*2 - 2)/2=56
            #nn.MaxPool1d(kernel_size, stride=None, padding=0,...) 
        )
        #重复的layer,分别有3， 4， 6， 3个residual block
        self.layer1=self._make_layer(64, 64, 3)#[56, 56, 64]
        self.layer2=self._make_layer(64, 128, 4, stride=2)#[28, 28, 128]
        self.layer3=self._make_layer(128, 256, 6, stride=2)#[14, 14, 256]
        self.layer4=self._make_layer(256, 512, 3, stride=2)#[7, 7, 512]
        #分类用的全连接
        self.fc=nn.Linear(512, num_classes)
    def _make_layer(self, in_channel, out_channel, block_num, stride=1):
        shortcut=nn.Sequential(
            nn.Conv2d(in_channel, out_channel, 1, stride, bias=False),#(56 - 1)/1 +1 =56   (56-1)/2+1=28
            nn.BatchNorm2d(out_channel)
        )
        layers=[]
        layers.append(ResidualBlock_2(in_channel, out_channel, stride, shortcut))
        for i in range(1, block_num):
            layers.append(ResidualBlock_2(out_channel, out_channel))
        return nn.Sequential(*layers)
    def forward(self, x):
        x=self.pre(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)
        x=F.avg_pool2d(x, 7)
        x=x.view(x.size(0), -1)
        return self.fc(x)

In [6]:
def ResNet(layer_num=18):
    if layer_num == 18:
        return ResNet_18(ResidualBlock_1)
    elif layer_num == 34:
        return ResNet_34()

In [7]:
print(ResNet(34))

ResNet_34(
  (pre): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): ResidualBlock_2(
      (left): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (right): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )


In [8]:
label2int = {'airplane':0, 'automobile':1, 'bird':2, 'cat':3, 'deer':4, 'dog':5, 'frog':6, 'horse':7, 'ship':8, 'truck':9}
int2lable = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [9]:
class TrainAndValData(Dataset):
    def __init__(self, img_path, csv_path, train=True, transforms=None):
        '''
        获得所有图片路径，并划分训练集、验证集
        '''
        self.train = train
        files = natsorted(glob.glob(img_path + '/*'))
        labels = pd.read_csv(csv_path).values[:, 1]
        files_num = len(files)
        break_point = int(0.9*files_num)
        if self.train:
            self.img_name = files[: break_point]
            self.img_label = labels[: break_point]
        else:
            self.img_name = files[break_point: ]
            self.img_label = labels[break_point: ]
         
   

        #数据增强
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            #训练集用数据增强
            if self.train:
                self.transforms = T.Compose([
                    #T.RandomCrop(224, padding=4),  #先四周填充0，在吧图像随机裁剪成32*32`，
                    T.Resize(224),
                    T.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
                    T.RandomVerticalFlip(),
                    T.ToTensor(),
                    normalize 
                ])
            else:
                self.transforms = T.Compose([
                    T.Resize(224),
                    #T.CenterCrop(224),#中心裁剪
                    T.ToTensor(),
                    normalize 
                ])
    def __len__(self):
        '''
        返回数据集中所有图片的个数
        '''
        return len(self.img_name)
    def __getitem__(self, index):
        '''
        返回一张图片的数据
        '''
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        label = label2int[self.img_label[index]]
        return img, label
class TestData(Dataset):
    def __init__(self, img_path, transforms=None):
        files = natsorted(glob.glob(img_path + '/*'))
        self.img_name = files
        if transforms is None:
            normalize = T.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
            self.transforms =  T.Compose([
                T.Resize(224),
                T.ToTensor(),
                normalize
            ])
    def __len__(self):
        return len(self.img_name)
    def __getitem__(self, index):
        img_path = self.img_name[index]
        img = Image.open(img_path)
        img = self.transforms(img)
        return img

In [10]:
train_img_path = 'data/train'
csv_path = 'data/trainLabels.csv'
test_img_path = 'data/test'
train_dataset = TrainAndValData(train_img_path, csv_path, train=True)
val_dataset = TrainAndValData(train_img_path, csv_path, train=False)
test_dataset = TestData(test_img_path)

In [11]:
print('train len is %d' % len(train_dataset))
print(train_dataset[0][0].shape)
print(train_dataset[0][1])#打印标签
print('val len is %d' % len(val_dataset))
print(val_dataset[0][0].shape)
print(val_dataset[0][1])
print('test len is %d' % len(test_dataset))
print(test_dataset[0].shape)

train len is 45000
torch.Size([3, 224, 224])
6
val len is 5000
torch.Size([3, 224, 224])
7
test len is 300000
torch.Size([3, 224, 224])


In [12]:
# 超参数设置
EPOCH = 135   #遍历数据集次数
BATCH_SIZE = 64      #批处理尺寸(batch_size)
#LR = 0.001        #学习率
lr = 0.01
lr_decay = 0.95
weight_decay = 1e-4
model_path = 'model/resnet/resnet_18.pkl'

In [13]:
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4) 

#device = torch.device("cuda:3")
#net = ResNet18()
net = ResNet(34)
net = net.cuda()
# 定义损失函数和优化方式
criterion = nn.CrossEntropyLoss()  #损失函数为交叉熵，多用于多分类问题
#optimizer = torch.optim.SGD(net.parameters(), lr=LR, momentum=0.9, weight_decay=5e-4) #优化方式为mini-batch momentum-SGD，并采用L2正则化（权重衰减）
optimizer = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)
#保存模型判断条件
max_val_acc = 0
pre_epoch = 0
max_interval_epoch = 10
pre_train_loss = 100000

print("Start Training...")
for epoch in range(100):
    #训练集
    train_loss = 0
    train_count = 0
    net.train()
    for i, data in enumerate(trainloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #更新指标
        train_count += labels.size(0)
        train_loss += loss.item()
    train_loss /= train_count

    #验证集
    val_acc = 0
    val_loss = 0
    val_count = 0
    net.eval()
    for i, data in enumerate(valloader):
        inputs, labels = data
        #inputs, labels = inputs.to(device), labels.to(device) # 注意需要复制到GPU
        inputs, labels =  inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        #更新指标
        val_count += 1
        val_loss += loss.item()
        _, predict = outputs.max(1)
        val_count += labels.size(0)
        val_acc += (predict == labels).sum().item()
    val_acc /= val_count
    val_loss /= val_count
    # print the loss and accuracy
    print('the epoch %d, the train loss is %f, the test loss is %f, the test acc is %f' % (epoch, train_loss, val_loss, val_acc))

    #保存模型
    if val_acc > max_val_acc:
        max_val_acc = val_acc
        pre_epoch = epoch
        torch.save(net, model_path)#保存整个神经网络的的结构信息和模型参数信息，save的对象是网络net
    if epoch - pre_epoch > max_interval_epoch:
        print('early stop')
        break

    #如果损失不载下降，则降低学习率
    if train_loss > pre_train_loss:
        lr = lr*lr_decay
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    pre_train_loss = pre_train_loss
print("Done Training!")

Start Training...
the epoch 0, the train loss is 0.027412, the test loss is 0.027113, the test acc is 0.399488


/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type ResNet_34. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/lhw/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type ResidualBlock_2. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


the epoch 1, the train loss is 0.020464, the test loss is 0.019289, the test acc is 0.543611
the epoch 2, the train loss is 0.016611, the test loss is 0.019076, the test acc is 0.568616
the epoch 3, the train loss is 0.014712, the test loss is 0.014722, the test acc is 0.651112
the epoch 4, the train loss is 0.013265, the test loss is 0.015915, the test acc is 0.632211
the epoch 5, the train loss is 0.012083, the test loss is 0.013253, the test acc is 0.694231
the epoch 6, the train loss is 0.011004, the test loss is 0.010805, the test acc is 0.742863
the epoch 7, the train loss is 0.010267, the test loss is 0.010526, the test acc is 0.748179
the epoch 8, the train loss is 0.009543, the test loss is 0.009820, the test acc is 0.770230
the epoch 9, the train loss is 0.008910, the test loss is 0.009051, the test acc is 0.780075
the epoch 10, the train loss is 0.008441, the test loss is 0.008581, the test acc is 0.794448
the epoch 11, the train loss is 0.007941, the test loss is 0.008900, 

Process Process-682:
Process Process-684:
Process Process-681:
Process Process-683:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/lhw/anaconda3/lib/python3.6/multiprocessing/process.py",

KeyboardInterrupt: 